In [174]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor

from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression

In [105]:
melbourne_data=pd.read_csv('cleaned_melbourne_housing.csv')

In [106]:
#extract year and day from date to capture time related patterns
melbourne_data['Date'] = pd.to_datetime(melbourne_data['Date'], format='%d/%m/%Y')

# Extract year and month as new columns
melbourne_data['Year'] = melbourne_data['Date'].dt.year
melbourne_data['Month'] = melbourne_data['Date'].dt.month

# Drop the original 'Date' column as it's no longer needed
melbourne_data = melbourne_data.drop(columns=['Date'])

# Display the updated dataset
melbourne_data.head()

,Suburb,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,Lattitude,Longtitude,Propertycount,log_price,Year,Month
0,Abbotsford,2,h,1480000.0,2.5,3067,1.0,1.0,202.0,-37.7996,144.9984,4019.0,6.170262,2016,12
1,Abbotsford,2,h,1035000.0,2.5,3067,1.0,0.0,156.0,-37.8079,144.9934,4019.0,6.014940,2016,2
2,Abbotsford,3,h,1465000.0,2.5,3067,2.0,0.0,134.0,-37.8093,144.9944,4019.0,6.165838,2017,3
3,Abbotsford,3,h,850000.0,2.5,3067,2.0,1.0,94.0,-37.7969,144.9969,4019.0,5.929419,2017,3
4,Abbotsford,4,h,1600000.0,2.5,3067,1.0,2.0,120.0,-37.8072,144.9941,4019.0,6.204120,2016,6


In [108]:
# Perform one-hot encoding for categorical variables
df_encoded = pd.get_dummies(melbourne_data, columns=['Suburb', 'Type'], drop_first=True)

In [109]:
# Define the target variable (log_price) and features (drop Price and log_price)
X = df_encoded.drop(columns=['Price', 'log_price'])
#set target as log_price
y = df_encoded['log_price']

In [110]:
# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



### MLR

### Baseline model (MLR with all features)

In [111]:
# Add a constant term for the intercept in statsmodels
X_train_const = sm.add_constant(X_train)

/Users/yuht/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [112]:
baseline_model = sm.OLS(y_train, X_train_const).fit()
print(baseline_model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.777
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     194.8
Date:                Fri, 25 Oct 2024   Prob (F-statistic):               0.00
Time:                        17:36:27   Log-Likelihood:                 15582.
No. Observations:               17383   AIC:                        -3.055e+04
Df Residuals:                   17076   BIC:                        -2.817e+04
Df Model:                         306                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [113]:
# Baseline Model Prediction and MSE Calculation
X_test_const = sm.add_constant(X_test)
baseline_predictions = baseline_model.predict(X_test_const)

y_test_original = np.exp(y_test)  # Original price values
baseline_predictions_original = np.exp(baseline_predictions)

baseline_mse = mean_squared_error(y_test_original, baseline_predictions_original)


/Users/yuht/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [114]:
baseline_mse

1617.4032896312297

### MLR model with stepwise feature selection using AIC/BIC

In [116]:
# Stepwise Feature Selection (AIC/BIC)
def stepwise_selection(X, y, criterion='aic'):
    """Perform a stepwise selection based on AIC or BIC."""
    X = sm.add_constant(X)  # Add constant for intercept
    included = []           # Track included features
    
    while True:
        changed = False
        
        # Forward step: try adding each feature
        excluded = list(set(X.columns) - set(included) - {'const'})
        new_aic_bic_scores = {}
        
        for new_column in excluded:
            model = sm.OLS(y, X[included + [new_column]]).fit()
            score = model.aic if criterion == 'aic' else model.bic
            new_aic_bic_scores[new_column] = score
            
        best_new_feature = min(new_aic_bic_scores, key=new_aic_bic_scores.get, default=None)
        best_score_with_new = new_aic_bic_scores.get(best_new_feature, float('inf'))
        
        # Backward step: try removing each feature
        aic_bic_scores_without = {}
        
        for column in included:
            model = sm.OLS(y, X[[col for col in included if col != column] + ['const']]).fit()
            score = model.aic if criterion == 'aic' else model.bic
            aic_bic_scores_without[column] = score
        
        best_remove_feature = min(aic_bic_scores_without, key=aic_bic_scores_without.get, default=None)
        best_score_without = aic_bic_scores_without.get(best_remove_feature, float('inf'))
        
        # Compare scores to add or remove
        if best_score_with_new < baseline_model.aic if criterion == 'aic' else baseline_model.bic:
            included.append(best_new_feature)
            changed = True
        elif best_score_without < baseline_model.aic if criterion == 'aic' else baseline_model.bic:
            included.remove(best_remove_feature)
            changed = True
        
        if not changed:
            break
            
    return included

# Apply Stepwise Selection
selected_features_aic = stepwise_selection(X_train, y_train, criterion='aic')
#selected_features_bic = stepwise_selection(X_train, y_train, criterion='bic')

print("Selected Features (AIC):", selected_features_aic)
#print("Selected Features (BIC):", selected_features_bic)

# Fit the final models using selected features
X_train_aic = sm.add_constant(X_train[selected_features_aic])
#X_train_bic = sm.add_constant(X_train[selected_features_bic])

final_model_aic = sm.OLS(y_train, X_train_aic).fit()
#final_model_bic = sm.OLS(y_train, X_train_bic).fit()

print("Final MLR Model Summary (AIC-based):")
print(final_model_aic.summary())
#print("Final MLR Model Summary (BIC-based):")
#print(final_model_bic.summary())

Selected Features (AIC): []
Final MLR Model Summary (AIC-based):
                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 25 Oct 2024   Prob (F-statistic):                nan
Time:                        17:37:15   Log-Likelihood:                 2525.7
No. Observations:               17383   AIC:                            -5049.
Df Residuals:                   17382   BIC:                            -5042.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [117]:
# AIC-selected Model Prediction and MSE Calculation
X_test_aic = sm.add_constant(X_test[selected_features_aic])
aic_predictions = final_model_aic.predict(X_test_aic)
y_test_original = np.exp(y_test)  # Original price values
aic_predictions_original = np.exp(aic_predictions)

aic_mse = mean_squared_error(y_test_original, aic_predictions_original)

# # BIC-selected Model Prediction and MSE Calculation
# X_test_bic = sm.add_constant(X_test[selected_features_bic])
# bic_predictions = final_model_bic.predict(X_test_bic)
# y_test_original = np.exp(y_test)  # Original price values
# bic_predictions_original = np.exp(bic_predictions)

# bic_mse = mean_squared_error(y_test_original, bic_predictions_original)


/Users/yuht/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [118]:
aic_mse

7097.400866234972

In [ ]:
#AIC is calculated based on the log-transformed target (log_price), it’s likely producing a high initial AIC value due to the smaller scale of the target, leading the model to not add any features. This smaller scale means that even minor additions in complexity may not appear to improve the AIC significantly enough for any features to be selected.

In [ ]:
#consider other global methods such as LASSO for feature selection

## Tree models

### RF

In [54]:
# Set up the parameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 10]
}
#5 fold cv
rf_grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                              param_grid=rf_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)


In [56]:
# Fit the model with cross-validation
rf_grid_search.fit(X_train, y_train)

# Best Random Forest Model
best_rf_model = rf_grid_search.best_estimator_

# Make predictions with the best Random Forest model
rf_predictions = best_rf_model.predict(X_test)

y_test_original = np.exp(y_test)  # Original price values
rf_predictions_original = np.exp(rf_predictions)  # RF predictions on original scale

rf_mse = mean_squared_error(y_test_original, rf_predictions_original)

/Users/yuht/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [58]:
rf_mse

1118.4143873881235

In [135]:
best_rf_model

RandomForestRegressor(min_samples_split=10, n_estimators=200, random_state=42)

In [137]:
#feature select with grid search cv
# Define pipeline with feature selection and Random Forest
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42))),
    ('rf', RandomForestRegressor(random_state=42))
])

# Define the parameter grid
param_grid = {
    'feature_selection__threshold': ['mean', 'median', 0.01],  # Adjust feature importance threshold
    'rf__n_estimators': [200],
    'rf__max_depth':  [None],
    'rf__min_samples_split': [10]
}

# Set up GridSearchCV with cross-validation on the pipeline
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=3,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1,
                           verbose=1)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Retrieve the best model and evaluate on the test set
best_selected_rf_model = grid_search.best_estimator_
rf_selected_predictions = best_selected_rf_model.predict(X_test)
y_test_original = np.exp(y_test)  # Original price values
rf_selected_predictions_original = np.exp(rf_selected_predictions)  # RF predictions on original scale

rf_selected_mse = mean_squared_error(y_test_original, rf_selected_predictions_original)


Fitting 3 folds for each of 3 candidates, totalling 9 fits


In [139]:
rf_selected_mse

1117.9916730642026

### XGBoost

In [59]:
# Set up the parameter grid for XGBoost
xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Set up the GridSearchCV for XGBoost
xgb_grid_search = GridSearchCV(estimator=XGBRegressor(random_state=42),
                               param_grid=xgb_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model with cross-validation
xgb_grid_search.fit(X_train, y_train)

# Best XGBoost Model
best_xgb_model = xgb_grid_search.best_estimator_

# Make predictions with the best XGBoost model
xgb_predictions = best_xgb_model.predict(X_test)

y_test_original = np.exp(y_test)  # Original price values
xgb_predictions_original = np.exp(xgb_predictions)

xgb_mse = mean_squared_error(y_test_original, xgb_predictions_original)

/Users/yuht/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [60]:
xgb_mse

1064.3567202013887

In [140]:
best_xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [143]:
#feature selection with grid search cv to optimize
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(XGBRegressor(n_estimators=100, random_state=42))),
    ('xgb', XGBRegressor(random_state=42))
])

# Define the parameter grid
param_grid = {
    'feature_selection__threshold': ['mean', 'median', 0.01],  # Thresholds for feature selection
    'xgb__n_estimators': [200],
    'xgb__max_depth': [6],
    'xgb__learning_rate': [0.2]
}

# Set up GridSearchCV with cross-validation on the pipeline
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1,
                           verbose=1)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Retrieve the best model and evaluate on the test set
best_selected_xgb_model = grid_search.best_estimator_
xgb_selected_predictions = best_selected_xgb_model.predict(X_test)
y_test_original = np.exp(y_test)  # Original price values
xgb_selected_predictions_original = np.exp(xgb_selected_predictions)

xgb_selected_mse = mean_squared_error(y_test_original, xgb_selected_predictions_original)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [144]:
xgb_selected_mse

1044.145553922678

### Adaboost

In [155]:
ada_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1]
}

# Set up the GridSearchCV for AdaBoost
ada_grid_search = GridSearchCV(estimator=AdaBoostRegressor(random_state=42),
                               param_grid=ada_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model with cross-validation
ada_grid_search.fit(X_train, y_train)

# Best AdaBoost Model
best_ada_model = ada_grid_search.best_estimator_

# Make predictions with the best AdaBoost model
ada_predictions = best_ada_model.predict(X_test)

y_test_original = np.exp(y_test)  # Original price values
ada_predictions_original = np.exp(xgb_predictions)

ada_mse = mean_squared_error(y_test_original, ada_predictions_original)

/Users/yuht/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [158]:
ada_mse

1064.3567202013887

In [145]:
best_ada_model

AdaBoostRegressor(learning_rate=0.1, n_estimators=100, random_state=42)

In [153]:
#feature selection with hyperparameter tuning
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(XGBRegressor(n_estimators=100,random_state=42))),  # Use RF to select features
    ('ada', AdaBoostRegressor(random_state=42))
])

# Define the parameter grid
param_grid = {
    'feature_selection__threshold': ['mean', 'median', 0.01],  # Thresholds for feature selection
    'ada__n_estimators': [100],
    'ada__learning_rate': [0.1]
}

# Set up GridSearchCV with cross-validation on the pipeline
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1,
                           verbose=1)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Retrieve the best model and evaluate on the test set
best_selected_ada_model = grid_search.best_estimator_
ada_selected_predictions = best_selected_ada_model.predict(X_test)

y_test_original = np.exp(y_test)  # Original price values
ada_selected_predictions_original = np.exp(ada_selected_predictions)

ada_selected_mse = mean_squared_error(y_test_original, ada_selected_predictions_original)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [154]:
ada_selected_mse

2770.238288243517

## Ensemble

### Stacking

In [163]:
best_xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [164]:
best_rf_model

RandomForestRegressor(min_samples_split=10, n_estimators=200, random_state=42)

In [165]:
best_ada_model

AdaBoostRegressor(learning_rate=0.1, n_estimators=100, random_state=42)

In [169]:
base_models = [
    ('rf', RandomForestRegressor(n_estimators=200,min_samples_split=10, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=200,max_depth=6,learning_rate=0.2, random_state=42)),
    ('ada', AdaBoostRegressor(n_estimators=100,learning_rate=0.1, random_state=42))
]

# Define the meta-model
meta_model = LinearRegression()

# Create the stacking model
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5)

# Train the stacking model
stacking_model.fit(X_train, y_train)

stacking_predictions = stacking_model.predict(X_test)
y_test_original = np.exp(y_test)  # Original price values
stacking_predictions_original = np.exp(stacking_predictions)

stacking_mse = mean_squared_error(y_test_original, stacking_predictions_original)

In [170]:
stacking_mse

1034.9332518187186

### Blending

In [176]:
rf_model = RandomForestRegressor(n_estimators=200,min_samples_split=10, random_state=42)
xgb_model = XGBRegressor(n_estimators=200,max_depth=6,learning_rate=0.2, random_state=42)
ada_model = AdaBoostRegressor(n_estimators=100,learning_rate=0.1, random_state=42)

rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
ada_model.fit(X_train, y_train)

# Get predictions from each model
rf_predictions = rf_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
ada_predictions = ada_model.predict(X_test)

# Simple average blending
blending_predictions = (rf_predictions + xgb_predictions + ada_predictions) / 3

y_test_original = np.exp(y_test)  # Original price values
blending_predictions_original = np.exp(blending_predictions)

blending_mse = mean_squared_error(y_test_original, blending_predictions_original)

In [177]:
blending_mse

1269.1948557494186

In [183]:
results = [
    {'Model': 'Baseline multiple linear regression', 'Description': 'MLR with all features', 'MSE': baseline_mse},
    {'Model': 'Optimized Random Forest', 'Description': 'RF with GridSearchCV', 'MSE': rf_mse},
    {'Model': 'RF with Feature Selection', 'Description': 'RF + SelectFromModel + GridSearchCV', 'MSE': rf_selected_mse},
    {'Model': 'Optimized XGBoost', 'Description': 'XGBoost with GridSearchCV', 'MSE': xgb_mse},
    {'Model': 'XGBoost with Feature Selection', 'Description': 'XGBoost + SelectFromModel + GridSearchCV', 'MSE': xgb_selected_mse},
    {'Model': 'Optimized AdaBoost', 'Description': 'AdaBoost with GridSearchCV', 'MSE': ada_mse},
    {'Model': 'Optimized Stacking', 'Description': 'RF + XGBoost + Adaboost', 'MSE': stacking_mse},
    {'Model': 'Optimized Blending', 'Description': '(RF + XGBoost + Adaboost) Predictions / 3', 'MSE': blending_mse}
]

# Convert results into a DataFrame
results_df = pd.DataFrame(results)
results_df.sort_values('MSE')

,Model,Description,MSE
6,Optimized Stacking,RF + XGBoost + Adaboost,1034.933252
4,XGBoost with Feature Selection,XGBoost + SelectFromModel + GridSearchCV,1044.145554
3,Optimized XGBoost,XGBoost with GridSearchCV,1064.356720
5,Optimized AdaBoost,AdaBoost with GridSearchCV,1064.356720
2,RF with Feature Selection,RF + SelectFromModel + GridSearchCV,1117.991673
1,Optimized Random Forest,RF with GridSearchCV,1118.414387
7,Optimized Blending,(RF + XGBoost + Adaboost) Predictions / 3,1269.194856
0,Baseline multiple linear regression,MLR with all features,1617.403290
